## Exploring different Document Processing Models
 
 **Goal**: An Agentic approach to explore different Document Processing Models for OCR, Layout Analysis, and Table Extraction.
 
 **Output**: A simple langgraph agent that explores different ocr techniques/models.


In [39]:
# Setting up a react agent with LangChain


from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.messages import AIMessageChunk, HumanMessage
from langchain_core.runnables import RunnableConfig

# from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.memory import InMemorySaver
from langsmith import traceable
from prompts import DOCUMENT_AGENT_SYSTEM_PROMPT, USER_MESSAGE_TEMPLATE

load_dotenv()
# conn = connect("langgraph_checkpoints.db")
# checkpointer = SqliteSaver(conn=conn)
checkpointer = InMemorySaver()
llm = init_chat_model(
    "anthropic:claude-sonnet-4-5-20250929", temperature=1, thinking={"type": "enabled", "budget_tokens": 10000}
)

document_agent = create_agent(
    model=llm,
    system_prompt=DOCUMENT_AGENT_SYSTEM_PROMPT,
    checkpointer=checkpointer,
)
config = RunnableConfig(configurable={"thread_id": "test_thread"})
inputs = {
    "messages": [
        HumanMessage(USER_MESSAGE_TEMPLATE.format(message="Hello, world!", document_path="No Document Uploaded"))
    ]
}


@traceable(name="DocumentAgentInteraction")
def agent_execution():
    """Execute the document agent and streams the response."""
    ai_message = AIMessageChunk(content=[])

    for _, chunk in document_agent.stream(inputs, stream_mode=["messages"], config=config):
        ai_message += chunk[0]

    ai_message.pretty_print()
    return ai_message


agent_execution()

============================ Aimessagechunk Message ============================

[{'thinking': 'The user has greeted me with "Hello, world!" and the system message indicates that no document has been uploaded yet. However, the instruction mentions that the user wants to process a document using all available OCR techniques.\n\nSince no document path is provided (the system message states "No Document Uploaded"), I cannot perform OCR yet. I should:\n1. Greet the user\n2. Acknowledge their request\n3. Inform them that no document has been uploaded yet\n4. Ask them to upload a document so I can proceed with OCR processing\n\nI should be helpful and ready to assist once they upload a document.', 'type': 'thinking', 'index': 0, 'signature': 'EvUFCkYICxgCKkAS1pcQGKQ6tP6RGDLwaMG5ok/5daHCKJj7+63FHQsJZU754Owx5VnS9YwtaoTg6yKK9Jxhyhc2y3rhzWHhM5/1Egwq/HlqXVRrRbo9eI8aDBN2XqVgFMxM8yM0mSIwkaDOA6RlzzIObJfd1Z3KYy5D+x0e7nHLUWGK2Fzp+MDliNDZJDWIkNJYT3BrjKtkKtwEYHbMWm44AL0T/9Ns1/w968gWf19LGt2/nSW7D5UtdjV4

AIMessageChunk(content=[{'thinking': 'The user has greeted me with "Hello, world!" and the system message indicates that no document has been uploaded yet. However, the instruction mentions that the user wants to process a document using all available OCR techniques.\n\nSince no document path is provided (the system message states "No Document Uploaded"), I cannot perform OCR yet. I should:\n1. Greet the user\n2. Acknowledge their request\n3. Inform them that no document has been uploaded yet\n4. Ask them to upload a document so I can proceed with OCR processing\n\nI should be helpful and ready to assist once they upload a document.', 'type': 'thinking', 'index': 0, 'signature': 'EvUFCkYICxgCKkAS1pcQGKQ6tP6RGDLwaMG5ok/5daHCKJj7+63FHQsJZU754Owx5VnS9YwtaoTg6yKK9Jxhyhc2y3rhzWHhM5/1Egwq/HlqXVRrRbo9eI8aDBN2XqVgFMxM8yM0mSIwkaDOA6RlzzIObJfd1Z3KYy5D+x0e7nHLUWGK2Fzp+MDliNDZJDWIkNJYT3BrjKtkKtwEYHbMWm44AL0T/9Ns1/w968gWf19LGt2/nSW7D5UtdjV41HS2/9rpOSZViSDUsLMrKc7vRXjeaTvLoAHwfiRxH3SukEPjcucguq3yG3K